# Elliptic Example: 2D Laplacian

We want to model the electric potential for an empty 2D box, 10cm x 10cm in size, where the top wall is held at $V$ = 1.0V and the other walls at 0V.

$$0 = \nabla^2 \phi  = \frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2},$$
$$ \phi(y = 10) = 1.0 V$$
$$ \phi(y = 0) = \phi(x = 0) = \phi(x = 10) = 0$$

![Newman fig. 9.2](fig9-2.png)

## Exercise 1

Setup up the problem: 
* discretize space in x and y, using an MxM grid
* implement the boundary conditions

Then use Jacobi Relaxation to solve it, with target accuracy 1e-04 and M=10. Print the number of iterations required to reach the target accuracy.

You can consult the textbook's `laplace.py` for help.

## Exercise 2

Plot the solution (you can use matplotlib.pyplot.imshow)

## Exercise 3

Now repeat Exercises 1 and 2 with N=100. Do you notice a difference in runtime?

## Exercise 4

Now repeat Exercise 3 with Gauss-Sidel instead of Jacobi method. You should get the same result, but with fewer iterations.

## Exercise 5

Now repeat with over-relaxation ($\omega = 0.9$) in the Gauss-Sidel.

# Hyperbolic Example: Wave Equation

Recall the 1D wave equation:
\begin{equation}
\frac{\partial^2\phi}{\partial t^2}=v^2\frac{\partial^2\phi}{\partial
  x^2}.\label{phieqn}\end{equation}
  
Consider a piano string of length $L$, initially at rest. At time $t=0$ the string is struck by the piano hammer a distance $d$ from the end of from the string. The string vibrates as a result of being struck, except at the ends, $x=0$, and $x=L$, where it is held fixed.

Consider the case $v=100\mathrm{ms^{-1}}$, with the initial condition that $\phi(x)=0$ everywhere but the velocity $\psi(x)$ is nonzero, with profile
\begin{equation}
  \psi(x) = C\frac{x(L-x)}{L^2}\exp\left[-\frac{(x-d)^2}{2\sigma^2}\right],
\end{equation}
where $L=1$m, $d=10$cm, $C=1\mathrm{ms^{-1}}$, and $\sigma=0.3$m. 

## Exercise 6

Solve using the FTCS method, with grid spacing (in $x$) $a=5$ nm, from times 0 to 0.01 s using time--step $h=10^{-6}$ s.
Make a plot of $\phi$ vs $x$ over the entire length of string, at each of the following times:
* 0.006 s
* 0.004 s
* 0.002 s
* 0.012 s
* 0.100 s

You'll see your first 4 plots look good, then the instability of the solution shows up!

## Exercise 7

Repeat the previous exercise using the Crank--Nicolson method. Use a larger time--step, $h = 10^{-4}$ s.

You'll see the solution is stable. It dies out to 0 at about 0.1 s, but this is how the physical system is supposed to behave!

The following snippets of code will help you define the matrix, and the vector to use on the right-hand side, of the CK equations.

In [13]:
from numpy import copy

def matrix(N,alpha):
    """Banded matrix for the Crank-Nicolson
    Args: 
        N : number of elements
        alpha = 2*h*v**2/a**2"""
    bands = np.zeros((3,N+2))
    bands[0,:-2] = -alpha
    bands[2,1:-1] = -alpha
    bands[1,:] = 1+2*alpha
    return bands 

def banded(Aa,va,up=1,down=1):
# from textbook online resources, to solve Ax = v
# Aa is banded matrix A, va is vector v, up and down give band positions in matrix

    # Copy the inputs and determine the size of the system
    A = copy(Aa)
    v = copy(va)
    N = len(v)

    # Gaussian elimination
    for m in range(N):

        # Normalization factor
        div = A[up,m]

        # Update the vector first
        v[m] /= div
        for k in range(1,down+1):
            if m+k<N:
                v[m+k] -= A[up+k,m]*v[m]

        # Now normalize the pivot row of A and subtract from lower ones
        for i in range(up):
            j = m + up - i
            if j<N:
                A[i,j] /= div
                for k in range(1,down+1):
                    A[i+k,j] -= A[up+k,m]*A[i,j]

    # Backsubstitution
    for m in range(N-2,-1,-1):
        for i in range(up):
            j = m + up - i
            if j<N:
                v[m] -= A[i,j]*v[j]

    return v

In [ ]:
def rhs(phi, psi, alpha,h):
    """Solve the Right hand side of the Crank-Nicolson algorithm.
    Args:
        phi, psi : position and velocity
        alpha = 2*h*v**2/a**2,
        h=dt, timestep
    
    Returns:
    the column vector for the right hand side.
    """
    r = np.zeros_like(phi)
    r[1:-1] = (h*psi[1:-1] + 
               alpha * phi[:-2] +
               (1-2*alpha)*phi[1:-1] + 
               alpha * phi[2:])
    return r

In [ ]:
alpha = h**2*v**2/(4*a**2)
A_mat = matrix(len(phi), alpha)
phiN = banded(A_mat, rhs(phi,psi,alpha,h))
psiN = (2./h)*(phiN-phi)-psi